In [11]:
!pip -q install transformers scikit-learn pandas numpy nltk tqdm joblib


*   !pip: The ! symbol allows you to run shell commands directly from a notebook cell.
*   -q: This is the "quiet" flag, which reduces the amount of output text during installation, showing only errors.
*   transformers: Provided by Hugging Face, this library is essential for using state-of-the-art machine learning models, especially for Natural Language Processing (NLP).
*  scikit-learn: A fundamental library for traditional machine learning in Python. It provides tools for data preprocessing, classification, regression, and model evaluation.
*   pandas: Used for data manipulation and analysis. It provides the DataFrame, a powerful data structure for handling tabular data like CSV files.
*   numpy: The core library for numerical computing in Python. It's crucial for performing mathematical operations on arrays and matrices.
*   nltk (Natural Language Toolkit): A library for various NLP tasks, such as splitting text into sentences or words (tokenization).
*  tqdm: A utility that creates smart progress bars for loops, making it easy to see the progress of long-running operations.
*   joblib: Used for efficiently saving and loading Python objects, which is perfect for storing trained machine learning models.





________________________________________


In [12]:
import os, re, json, math, gc, random
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
import joblib

import nltk
nltk.download('punkt')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.utils.class_weight import compute_class_weight

import torch
from transformers import pipeline, logging as hf_logging
hf_logging.set_verbosity_error()

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

DEVICE = 0 if torch.cuda.is_available() else -1
print("Using device:", "GPU" if DEVICE == 0 else "CPU")

os.makedirs('models', exist_ok=True)
os.makedirs('outputs', exist_ok=True)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using device: CPU


*   These lines import standard Python libraries for file operations (os, Path), regular expressions (re), handling JSON data (json), math functions (math), garbage collection (gc), and generating random numbers (random). numpy and pandas are imported with their standard aliases np and pd. tqdm and joblib are also imported.
*   The nltk (NATURAL LANGUAGE TOOLKIT) library is imported.
*   nltk.download('punkt'): This command downloads the 'punkt' tokenizer model, which is pre-trained to split text into sentences effectively.
*   This section imports specific tools from scikit-learn:
1.   train_test_split: A function to split data into training and testing sets.
1.   TfidfVectorizer: A tool to convert a collection of text documents into a matrix of TF-IDF (TERM FREQUENCY INVERSE DOCUMENT FREQUENCY) features.
1.   LogisticRegression, SGDClassifier: Two different algorithms for classification.
2.   classification_report: A function to build a text report showing the main classification metrics.
2.   Pipeline: A tool to chain multiple steps together (e.g., vectorization and classification) into a single workflow.
2.  compute_class_weight: A utility to handle imbalanced datasets by giving more weight to under-represented classes.

*   torch: Imports the PyTorch library, which is the backend for the transformers models.
*  The pipeline function from transformers is a high-level, easy-to-use API for running models.
*   The logging level for transformers is set to show only errors, keeping the output clean

*   RANDOM_STATE = 42: A fixed seed is set for all random operations. This ensures that results are reproducible; anyone running the notebook will get the exact same train/test splits and model initializations.
*   np.random.seed(RANDOM_STATE):
This line sets the seed for numpy's pseudo-random number generator. When a seed is provided, numpy will produce the same sequence of "random" numbers every time the code is executed with that specific seed. This ensures reproducibility in operations involving numpy's random functions, such as data splitting, array shuffling, or initializations in machine learning algorithms.
*   random.seed(RANDOM_STATE):
Similarly, this line sets the seed for Python's built-in random module. This ensures that any subsequent calls to functions within the random module (e.g., random.randint(), random.choice()) will also produce a reproducible sequence of "random" numbers.
*  This code checks if a CUDA(COMPUTE UNIFIED DEVICE ARCHITECTURE)-enabled GPU is available for PyTorch. If yes, DEVICE is set to 0 (the first GPU). If not, it's set to -1 (CPU). This allows the transformers pipeline to automatically use the faster GPU if possible.
*  These lines create two directories, models and outputs, if they don't already exist. This is where the trained models and resulting data files will be saved.




In [13]:
# Set the CSV path (change if needed)
DATA_PATH = '/content/drive/MyDrive/Hotels/hotels_reviews.csv'  # update to your file path

# ABSA runtime knobs
MAX_REVIEWS_FOR_ABSA = 2000   # increase later after validating results
SENT_MIN_LEN = 5              # minimum sentence length to consider
ASPECT_THRESHOLD = 0.45       # zero-shot score threshold (tune 0.35–0.55)
TOPK_PER_SENT = 3             # cap aspects per sentence


*   Sets the file path for the input dataset of hotel reviews.
*   MAX_REVIEWS_FOR_ABSA: Limits the number of reviews to process during the main analysis. This is useful for faster testing and validation before running on the full dataset.
*   SENT_MIN_LEN: Sets the minimum number of characters a sentence must have to be considered for analysis, filtering out very short, non-informative snippets.
*   ASPECT_THRESHOLD: A confidence score threshold for the initial keyword-based aspect detection method. (Note: This is later superseded by a different threshold in the BERT-based approach).
*   TOPK_PER_SENT: Limits the maximum number of aspects that can be detected in a single sentence to prevent noise.


In [14]:
ASPECTS = [
    # 1. Accommodation (The Room)
    "Cleanliness & Hygiene", "Comfort", "Room Amenities", "Bathroom",
    "Noise Level & Ambiance", "Space & Layout",
    # 2. Service & Staff
    "Check-in & Check-out", "Staff Professionalism & Attitude", "Responsiveness",
    "Concierge Knowledge", "Housekeeping Efficiency",
    # 3. Location & Accessibility
    "Proximity", "Neighborhood", "Accessibility & Parking", "The View",
    # 4. Food & Beverage
    "Restaurant Quality", "Breakfast", "Room Service", "Bar/Lounge",
    # 5. Facilities & Amenities
    "Recreational Facilities", "Common Areas", "Technology",
    "Business Services", "Accessibility Features",
    # 6. Overall Value & Atmosphere
    "Value for Money", "Ambiance & Design", "Safety & Security", "Sustainability Practices",
]

ASPECT_HINTS = {
    "Cleanliness & Hygiene": ["clean", "dirty", "hygiene", "spotless", "stains", "smell", "odor"],
    "Comfort": ["bed", "mattress", "pillow", "blanket", "comfortable", "uncomfortable"],
    "Room Amenities": ["wifi", "tv", "ac", "air conditioning", "coffee", "tea", "minibar", "safe", "amenities"],
    "Bathroom": ["bathroom", "shower", "toilet", "water pressure", "hot water", "toiletries", "towel"],
    "Noise Level & Ambiance": ["noise", "noisy", "quiet", "soundproof", "ambiance", "lighting", "curtains"],
    "Space & Layout": ["spacious", "space", "cramped", "layout", "storage", "outlet", "usb"],
    "Check-in & Check-out": ["check in", "check-in", "check out", "check-out", "reception"],
    "Staff Professionalism & Attitude": ["staff", "friendly", "rude", "polite", "courteous", "welcoming"],
    "Responsiveness": ["respond", "response", "quickly", "prompt", "slow", "waited"],
    "Concierge Knowledge": ["concierge", "recommendation", "booking", "tour", "transport"],
    "Housekeeping Efficiency": ["housekeeping", "cleaning", "turndown", "room service timing"],
    "Proximity": ["close to", "near", "distance", "walk", "minutes away"],
    "Neighborhood": ["area", "neighborhood", "surrounding", "nearby", "safety"],
    "Accessibility & Parking": ["parking", "accessible", "ramp", "elevator", "wheelchair"],
    "The View": ["view", "scenery", "overlook", "window"],
    "Restaurant Quality": ["restaurant", "dinner", "lunch", "meal", "taste", "menu", "chef"],
    "Breakfast": ["breakfast", "buffet", "morning"],
    "Room Service": ["room service", "in-room dining", "order"],
    "Bar/Lounge": ["bar", "lounge", "cocktail", "drink"],
    "Recreational Facilities": ["pool", "spa", "gym", "fitness", "sauna", "jacuzzi"],
    "Common Areas": ["lobby", "hall", "corridor", "public area"],
    "Technology": ["wifi", "internet", "speed", "connectivity"],
    "Business Services": ["meeting", "conference", "business center", "printing"],
    "Accessibility Features": ["accessible room", "grab bar", "elevator", "ramp"],
    "Value for Money": ["value", "price", "expensive", "overpriced", "worth", "deal"],
    "Ambiance & Design": ["design", "decor", "aesthetic", "ambience", "atmosphere"],
    "Safety & Security": ["security", "safe", "lock", "guard", "well-lit"],
    "Sustainability Practices": ["eco", "recycle", "conservation", "plastic"],
}

def hint_aspects_for_sentence(sent: str):
    s = sent.lower()
    hits = set()
    for aspect, hints in ASPECT_HINTS.items():
        if any(h in s for h in hints):
            hits.add(aspect)
    return list(hits)


*   This list defines the 28 distinct categories you want to analyze in the reviews, such as service, cleanliness, and value.
*  This dictionary maps each aspect to a list of keywords or "hints". For example, if a sentence contains the word "clean" or "dirty," it will be initially tagged with the "Cleanliness & Hygiene" aspect. This is a simple, keyword-based approach to aspect detection.
*  This function takes a sentence, converts it to lowercase, and checks if any of the keywords from ASPECT_HINTS are present. It returns a list of all aspects that were "hit" in that sentence.


In [15]:
def load_reviews(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)

    # Identify text column
    possible_text_cols = ["Review_Text", "review", "text", "content", "Review"]
    text_col = next((c for c in possible_text_cols if c in df.columns), None)
    if text_col is None:
        raise ValueError(f"Couldn't find a review text column in {list(df.columns)}")

    title_col = "Review_Title" if "Review_Title" in df.columns else None
    rating_col = "Review_Rating" if "Review_Rating" in df.columns else None

    # Combine title + text
    if title_col:
        df["text"] = (df[title_col].fillna("") + ". " + df[text_col].fillna("")).str.strip()
    else:
        df["text"] = df[text_col].fillna("")

    # Derive overall label from rating if present (50→5★ …)
    if rating_col:
        stars = (df[rating_col] / 10.0).round(0)
        df["overall_label"] = np.where(stars <= 2, "negative",
                                np.where(stars >= 4, "positive", "neutral"))
    else:
        df["overall_label"] = None

    # Drop empty rows
    df = df[df["text"].str.strip().str.len() > 0].reset_index(drop=True)
    return df

df = load_reviews(DATA_PATH)
print("Loaded:", df.shape)
df.head(3)


Loaded: (55368, 11)


,Hotel_Name,Review_Date,Review_Rating,Review_Title,Review_Text,Reviewer_Date_Of_Stay,Reviewer_Trip_Type,Reviewer_Location,Reviewer_Profile_Link,text,overall_label
0,72 Riad Living,Rachel R wrote a review Mar 23,50,Lovely ambience and service,I honestly couldn't fault Riad 72. The staff a...,Date of stay: March 2023,Trip type: Traveled as a couple,"Edinburgh, United Kingdom",https://www.tripadvisor.com/Profile/rachrichmond,Lovely ambience and service. I honestly couldn...,positive
1,72 Riad Living,Ron N wrote a review Feb 2023,50,Top marks well earned,We stayed for four nights at 72 Riad Living (w...,Date of stay: February 2023,NaN,"Banff, Canada",https://www.tripadvisor.com/Profile/Gadman,Top marks well earned. We stayed for four nigh...,positive
2,72 Riad Living,puddu101 wrote a review Mar 22,40,An Oasis,With a superb location it felt like an oasis a...,Date of stay: February 2023,NaN,"Mumbai, India",https://www.tripadvisor.com/Profile/puddu101,An Oasis. With a superb location it felt like ...,positive


*   This cell defines a function to load the data from the CSV file and perform initial cleaning and feature engineering.
*   Defines the function load_reviews and uses pandas to read the CSV file into a DataFrame df.
*   This code robustly finds the main review text column by checking against a list of common names (possible_text_cols). This makes the script more adaptable to different CSV file formats
*   It combines the review title and the review body into a single text column for a more complete analysis. .fillna("") handles cases where a title or text might be missing.
*   This section converts the numerical rating (e.g., 10-50) into a categorical sentiment label (positive, neutral, negative). It first normalizes ratings to a 1-5 star scale and then assigns labels based on thresholds (1-2 stars = negative, 3 = neutral, 4-5 = positive).
*   The function is called to load the data, and the shape (rows, columns) of the resulting DataFrame is printed. df.head(3) displays the first 3 rows for a quick preview.


In [ ]:
def train_overall_sentiment_baseline(df: pd.DataFrame):
    labeled = df.dropna(subset=["overall_label"]).copy()
    if labeled.empty:
        print("No ratings found; skipping baseline training.")
        return None

    X = labeled["text"].values
    y = labeled["overall_label"].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
    )

    classes = np.unique(y_train)
    weights = compute_class_weight("balanced", classes=classes, y=y_train)
    class_weight = {cls: w for cls, w in zip(classes, weights)}

    clf = Pipeline([
        ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.9, sublinear_tf=True)),
        ("logreg", LogisticRegression(max_iter=1000, class_weight=class_weight, solver="lbfgs"))
    ])
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    print("\n=== Overall Sentiment Baseline ===")
    print(classification_report(y_test, preds, digits=3))
    joblib.dump(clf, "models/lr_sentiment.joblib")
    print("Saved ➜ models/lr_sentiment.joblib")
    return clf

baseline_model = train_overall_sentiment_baseline(df)



=== Overall Sentiment Baseline ===
              precision    recall  f1-score   support

    negative      0.730     0.793     0.760       747
     neutral      0.455     0.703     0.552       900
    positive      0.985     0.927     0.956      9427

    accuracy                          0.900     11074
   macro avg      0.723     0.808     0.756     11074
weighted avg      0.925     0.900     0.910     11074

Saved ➜ models/lr_sentiment.joblib


This cell trains a simple, overall sentiment model. This model acts as a "teacher" to generate labels for individual sentences later on—a technique called weak supervision.

*  The function takes the DataFrame, drops rows without an overall label, and splits the data into 80% for training and 20% for testing. stratify=y ensures that the proportion of positive, neutral, and negative reviews is the same in both the training and testing sets.
*   This calculates class weights to address the imbalanced dataset (there are far more positive reviews than negative or neutral ones). The class_weight="balanced" setting in LogisticRegression will penalize mistakes on minority classes more heavily during training, helping the model learn them better.
*   A scikit-learn Pipeline is created. This chains two steps:
1.	"tfidf": A TfidfVectorizer converts the text into numerical vectors. ngram_range=(1,2) tells it to consider both single words (unigrams) and pairs of words (bigrams) as features.
2.	"logreg": A LogisticRegression model is used as the classifier, configured with the calculated class weights.
*  The pipeline is trained on the training data (.fit).
*  It makes predictions on the unseen test data (.predict).
*   A classification_report is printed to show the model's performance (precision, recall, F1-score).
*   Finally, the trained model is saved to a file using joblib so it can be loaded later without retraining.

In [ ]:
def init_incremental_sgd():
    return Pipeline([
        ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=2, max_df=0.95, sublinear_tf=True)),
        ("sgd", SGDClassifier(loss="log_loss", max_iter=5, random_state=RANDOM_STATE))
    ])

def partial_fit_incremental(model, texts, labels):
    classes = np.array(["negative","neutral","positive"])
    tfidf = model.named_steps["tfidf"]
    if not hasattr(tfidf, "vocabulary_"):
        X_t = tfidf.fit_transform(texts)
        model.named_steps["sgd"].partial_fit(X_t, labels, classes=classes)
    else:
        X_t = tfidf.transform(texts)
        model.named_steps["sgd"].partial_fit(X_t, labels)
    return model

# Example usage later once you collect human-verified labels:
# incr = init_incremental_sgd()
# incr = partial_fit_incremental(incr, labeled_texts, labeled_y)
# joblib.dump(incr, "models/incremental_sgd.joblib")


This cell defines functions for an SGDClassifier, which can be incrementally trained with new data over time (partial_fit). This is set up but not used in the main workflow.
*   These functions define a pipeline using SGDClassifier, which is suitable for online or incremental learning. This would be useful if we had a system where new, human-verified labels were arriving over time and you wanted to update the model without retraining from scratch.



In [ ]:
from nltk.tokenize import sent_tokenize
import joblib
from collections import Counter

BASELINE_PATH = "models/lr_sentiment.joblib"
baseline = joblib.load(BASELINE_PATH)  # your trained TF-IDF + Logistic Regression

def baseline_predict_label_conf(texts):
    """Returns labels and confidences from your baseline (applied to sentences)."""
    proba = baseline.predict_proba(texts)
    classes = baseline.classes_
    idx = proba.argmax(axis=1)
    labels = classes[idx]
    confs = proba.max(axis=1)
    return labels, confs

def extract_aspect_sentence_rows(df):
    rows = []
    for ridx, row in df.iterrows():
        text = str(row["text"])
        sents = [s.strip() for s in sent_tokenize(text) if len(s.strip()) >= SENT_MIN_LEN]
        for s in sents:
            hinted = hint_aspects_for_sentence(s)  # from Cell 5
            if not hinted:
                continue
            rows.append({"review_idx": int(ridx), "sentence": s, "aspects": hinted})
    return pd.DataFrame(rows)

sent_df = extract_aspect_sentence_rows(df)
print("Sentences with hinted aspects:", sent_df.shape)

# teacher labels from your baseline (applied at sentence-level)
sent_df["sentiment"], sent_df["sent_conf"] = baseline_predict_label_conf(sent_df["sentence"].tolist())

# explode to per-(sentence, aspect) rows for training per-aspect sentiment models
exploded = sent_df.explode("aspects").rename(columns={"aspects":"aspect"}).reset_index(drop=True)

# Save datasets for inspection
os.makedirs("outputs", exist_ok=True)
sent_df.to_csv("outputs/aspect_sentence_dataset.csv", index=False)
exploded.to_csv("outputs/aspect_sentence_dataset_exploded.csv", index=False)

print("Saved ➜ outputs/aspect_sentence_dataset.csv & _exploded.csv")
print(exploded.head())
print("\nClass balance (teacher labels):", Counter(exploded["sentiment"]))


Sentences with hinted aspects: (391966, 3)
Saved ➜ outputs/aspect_sentence_dataset.csv & _exploded.csv
   review_idx                                           sentence  \
0           0                       Lovely ambience and service.   
1           0  The staff are so nice (helpful and polite) the...   
2           0  The staff are so nice (helpful and polite) the...   
3           0  The staff are so nice (helpful and polite) the...   
4           0  The staff are so nice (helpful and polite) the...   

                             aspect sentiment  sent_conf  
0                 Ambiance & Design  positive   0.881578  
1                    Room Amenities  positive   0.955506  
2  Staff Professionalism & Attitude  positive   0.955506  
3                 Ambiance & Design  positive   0.955506  
4                         Proximity  positive   0.955506  

Class balance (teacher labels): Counter({'positive': 522296, 'negative': 84859, 'neutral': 77406})


This cell uses the "teacher" model from Cell 6 to label individual sentences. It first extracts all sentences that contain aspect keywords and then applies the baseline model to predict their sentiment.

*   The previously trained baseline ("teacher") model is loaded from the file.
*   This function iterates through every review, splits it into sentences, and for each sentence, uses the hint_aspects_for_sentence function (from Cell 4) to find aspect keywords. It creates a new DataFrame where each row is a sentence that contained at least one aspect hint.
*   The function is called to create sent_df.
*   The baseline model is then used to predict the sentiment for every one of these sentences. The predicted label and the model's confidence are stored in new columns. This is the core of the weak supervision process.
*    The sent_df is "exploded". If a sentence was tagged with multiple aspects (e.g., ["Comfort", "Cleanliness"]), this creates separate rows for each aspect, duplicating the sentence text and sentiment. This prepares the data for training individual models for each aspect



In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
import joblib

# X = sentence text; Y = multi-hot aspects from hints (weak labels)
X_text = sent_df["sentence"].tolist()
Y_list = sent_df["aspects"].tolist()

mlb = MultiLabelBinarizer(classes=ASPECTS)
Y = mlb.fit_transform(Y_list)

X_train, X_test, Y_train, Y_test = train_test_split(
    X_text, Y, test_size=0.2, random_state=RANDOM_STATE
)

vec = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.95, sublinear_tf=True)
Xtr = vec.fit_transform(X_train)
Xte = vec.transform(X_test)

ovr = OneVsRestClassifier(
    LogisticRegression(max_iter=1000, class_weight="balanced", solver="lbfgs")
)
ovr.fit(Xtr, Y_train)

Y_pred = ovr.predict(Xte)
print("Aspect detector micro-F1:", f1_score(Y_test, Y_pred, average="micro"))
print("Aspect detector macro-F1:", f1_score(Y_test, Y_pred, average="macro"))

joblib.dump({"vec": vec, "clf": ovr, "mlb": mlb}, "models/aspect_detector_ovr.joblib")
print("Saved ➜ models/aspect_detector_ovr.joblib")


Aspect detector micro-F1: 0.9850632367432521
Aspect detector macro-F1: 0.9823965261026092
Saved ➜ models/aspect_detector_ovr.joblib


These cells train the final models using the weakly-labeled data generated by the simple keyword-matching approach. Note: The "BERT FIX" section later replaces this entire approach with a much better one.
*   Cell 9: Trains the Aspect Detector. It's a OneVsRestClassifier wrapped around LogisticRegression. This is a multi-label classifier that learns to predict which aspect(s) are present in a sentence based on the TF-IDF features of the text.



In [ ]:
from sklearn.pipeline import Pipeline

def slugify(name: str):
    import re
    return re.sub(r'[^a-z0-9]+','_', name.lower()).strip('_')

min_rows_per_aspect = 80      # lower if your data is sparse
min_classes_per_aspect = 2    # require at least 2 classes to train

trained_aspect_models = {}

for aspect in ASPECTS:
    df_a = exploded[exploded["aspect"] == aspect].copy()
    if len(df_a) < min_rows_per_aspect:
        print(f"[skip] {aspect}: not enough sentences ({len(df_a)})")
        continue

    y = df_a["sentiment"]
    if y.nunique() < min_classes_per_aspect:
        print(f"[skip] {aspect}: only one sentiment class present")
        continue

    X_train, X_test, y_train, y_test = train_test_split(
        df_a["sentence"], y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
    )

    pipe = Pipeline([
        ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.95, sublinear_tf=True)),
        ("logreg", LogisticRegression(max_iter=1000, class_weight="balanced", solver="lbfgs"))
    ])
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)
    print(f"\n=== Sentiment model for [{aspect}] ===")
    print(classification_report(y_test, preds, digits=3))

    path = f"models/sentiment_aspect_{slugify(aspect)}.joblib"
    joblib.dump(pipe, path)
    trained_aspect_models[aspect] = path
    print("Saved ➜", path)

# Optional: general sentence-level student (fallback if an aspect model is missing)
gen_pipe = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.95, sublinear_tf=True)),
    ("logreg", LogisticRegression(max_iter=1000, class_weight="balanced", solver="lbfgs"))
])
gen_pipe.fit(sent_df["sentence"], sent_df["sentiment"])
joblib.dump(gen_pipe, "models/sentiment_sentence_student.joblib")
print("\nSaved general sentence sentiment student ➜ models/sentiment_sentence_student.joblib")



=== Sentiment model for [Cleanliness & Hygiene] ===
              precision    recall  f1-score   support

    negative      0.738     0.920     0.819       662
     neutral      0.704     0.835     0.764       763
    positive      0.987     0.919     0.952      4387

    accuracy                          0.908      5812
   macro avg      0.810     0.891     0.845      5812
weighted avg      0.922     0.908     0.912      5812

Saved ➜ models/sentiment_aspect_cleanliness_hygiene.joblib

=== Sentiment model for [Comfort] ===
              precision    recall  f1-score   support

    negative      0.646     0.860     0.738       543
     neutral      0.638     0.817     0.716       652
    positive      0.979     0.883     0.928      3731

    accuracy                          0.872      4926
   macro avg      0.754     0.854     0.794      4926
weighted avg      0.897     0.872     0.879      4926

Saved ➜ models/sentiment_aspect_comfort.joblib

=== Sentiment model for [Room Amenities

*   Cell 10: Trains the Per-Aspect Sentiment Models. It iterates through each of the 28 aspects. For each one, it filters the exploded DataFrame to get only the sentences for that aspect and trains a dedicated sentiment classifier on them. This creates specialized models (e.g., a "Breakfast" sentiment model, a "Staff" sentiment model).



In [ ]:
import joblib
import numpy as np

# thresholds for inference
ASPECT_PROBA_THRESHOLD = 0.50   # choose aspects with proba ≥ 0.5
ASPECT_TOPK = 3                 # also cap per sentence to top-K (set None to disable)

# load models
aspect_detect = joblib.load("models/aspect_detector_ovr.joblib")
aspect_vec = aspect_detect["vec"]
aspect_clf = aspect_detect["clf"]
aspect_mlb = aspect_detect["mlb"]

# load fallback student model
sent_student = joblib.load("models/sentiment_sentence_student.joblib")

# discover available per-aspect models
aspect_model_paths = {a: f"models/sentiment_aspect_{slugify(a)}.joblib" for a in ASPECTS}
aspect_models = {}
for a, p in aspect_model_paths.items():
    if os.path.exists(p):
        aspect_models[a] = joblib.load(p)

def predict_aspects_for_sentence(sent: str):
    X = aspect_vec.transform([sent])
    # predict_proba works because base estimator is LogisticRegression
    proba = aspect_clf.predict_proba(X)[0]  # shape [n_aspects]
    labels = np.array(aspect_mlb.classes_)
    # threshold + topK
    pairs = [(lab, float(p)) for lab, p in zip(labels, proba) if p >= ASPECT_PROBA_THRESHOLD]
    pairs.sort(key=lambda x: -x[1])
    if ASPECT_TOPK:
        pairs = pairs[:ASPECT_TOPK]
    return pairs

def predict_sentiment_for_sentence_aspect(sent: str, aspect: str):
    model = aspect_models.get(aspect, None)
    if model is None:
        # fallback to general student
        label = sent_student.predict([sent])[0]
        conf = max(sent_student.predict_proba([sent])[0])
        return label, float(conf)
    else:
        label = model.predict([sent])[0]
        conf = max(model.predict_proba([sent])[0])
        return label, float(conf)

def analyze_review(text: str):
    sents = [s.strip() for s in sent_tokenize(text) if len(s.strip()) >= SENT_MIN_LEN]
    aspect_map = {}
    for s in sents:
        asp_pairs = predict_aspects_for_sentence(s)
        if not asp_pairs:
            continue
        for (asp, _p) in asp_pairs:
            lab, conf = predict_sentiment_for_sentence_aspect(s, asp)
            prev = aspect_map.get(asp, {"votes": {"negative":0,"neutral":0,"positive":0}, "max_conf":0.0})
            prev["votes"][lab] += 1
            prev["max_conf"] = max(prev["max_conf"], conf)
            aspect_map[asp] = prev

    # finalize per-aspect label by majority vote
    finalized = {}
    for asp, v in aspect_map.items():
        label = max(v["votes"].items(), key=lambda kv: kv[1])[0]
        finalized[asp] = {
            "sentiment": label,
            "hits": sum(v["votes"].values()),
            "conf": round(v["max_conf"], 3)
        }
    return finalized


*   Cell 11: Sets up the Inference Pipeline by loading all the trained models and defining functions to use them. analyze_review ties everything together: it takes a review, splits it, uses the aspect detector to find aspects in each sentence, and then uses the correct per-aspect sentiment model to get the final sentiment.

In [ ]:
from tqdm import tqdm
import json

def run_absa_scikit(df: pd.DataFrame, max_reviews=None):
    work = df.copy()
    if max_reviews and len(work) > max_reviews:
        work = work.iloc[:max_reviews].copy()

    rows = []
    for i, row in tqdm(work.iterrows(), total=len(work), desc="ABSA (sklearn)"):
        text = str(row["text"])
        aspects = analyze_review(text)
        rows.append({
            "idx": i,
            "text": text,
            "aspects_json": json.dumps(aspects, ensure_ascii=False),
            "aspects_list": " | ".join(
                f"{a}={v['sentiment']} ({v['hits']} hits, conf≈{v['conf']:.2f})" for a, v in aspects.items()
            ) if aspects else "",
            "overall_label": row.get("overall_label", None)
        })

    out = pd.DataFrame(rows)
    out.to_csv("outputs/review_level_absa.csv", index=False)
    print("Saved ➜ outputs/review_level_absa.csv")

    # explode for summary
    long_rows = []
    for _, r in out.iterrows():
        d = json.loads(r["aspects_json"])
        for a, v in d.items():
            long_rows.append({
                "idx": r["idx"],
                "aspect": a,
                "sentiment": v["sentiment"],
                "hits": v["hits"],
                "conf": v["conf"],
            })

    if not long_rows:
        print("No aspect rows produced — check thresholds or training data.")
        return out, None, None

    long_df = pd.DataFrame(long_rows)
    long_df.to_csv("outputs/review_aspect_sentiments_long.csv", index=False)
    print("Saved ➜ outputs/review_aspect_sentiments_long.csv")

    summary = (long_df.groupby(["aspect","sentiment"])
               .size().reset_index(name="count")
               .pivot(index="aspect", columns="sentiment", values="count")
               .fillna(0).astype(int).reset_index().sort_values("aspect"))
    for col in ["negative","neutral","positive"]:
        if col not in summary.columns: summary[col] = 0
    summary = summary[["aspect","negative","neutral","positive"]]
    summary.to_csv("outputs/aspect_summary.csv", index=False)
    print("Saved ➜ outputs/aspect_summary.csv")

    return out, long_df, summary

# 🔹 run end-to-end (start with a small subset to validate, then scale)
review_level_absa, long_df, aspect_summary = run_absa_scikit(df, max_reviews=300)  # increase after validating
display(review_level_absa.head(3))
display(aspect_summary.head(10))


ABSA (sklearn): 100%|██████████| 300/300 [00:25<00:00, 11.64it/s]


Saved ➜ outputs/review_level_absa.csv
Saved ➜ outputs/review_aspect_sentiments_long.csv
Saved ➜ outputs/aspect_summary.csv


,idx,text,aspects_json,aspects_list,overall_label
0,0,Lovely ambience and service. I honestly couldn...,"{""Ambiance & Design"": {""sentiment"": ""positive""...","Ambiance & Design=positive (2 hits, conf≈0.95)...",positive
1,1,Top marks well earned. We stayed for four nigh...,"{""Staff Professionalism & Attitude"": {""sentime...",Staff Professionalism & Attitude=positive (1 h...,positive
2,2,An Oasis. With a superb location it felt like ...,"{""Staff Professionalism & Attitude"": {""sentime...",Staff Professionalism & Attitude=positive (1 h...,positive


sentiment,aspect,negative,neutral,positive
0,Accessibility & Parking,0,1,3
1,Accessibility Features,0,1,0
2,Ambiance & Design,0,2,68
3,Bar/Lounge,1,2,42
4,Bathroom,1,1,21
5,Breakfast,3,2,98
6,Business Services,0,0,3
7,Check-in & Check-out,4,0,8
8,Cleanliness & Hygiene,4,5,50
9,Comfort,1,2,66


*  Cell 12: Runs the End-to-End Pipeline on a sample of the data, generating the final output CSVs (review_level_absa.csv, etc.).


**BERT FIX OF CELLS 9-12**

In [ ]:
!pip -q install sentence-transformers nltk
import nltk
nltk.download('wordnet'); nltk.download('omw-1.4'); nltk.download('punkt')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

This section introduces a major improvement. The keyword-based method is rigid and can miss synonyms or contextual meaning. This new approach uses a powerful Sentence-BERT model to understand the semantic meaning of sentences, leading to much more accurate aspect detection.
*   Installs the sentence-transformers library and downloads NLTK's wordnet for finding synonyms.


In [ ]:
from sentence_transformers import SentenceTransformer, util
from nltk.corpus import wordnet as wn
import numpy as np
import re, json

# light & fast sentence embedding model
SBERT_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
sbert = SentenceTransformer(SBERT_MODEL_NAME)

# Aspect definitions (short descriptions help create better prototypes)
ASPECT_DESCRIPTIONS = {
    "Cleanliness & Hygiene": "clean rooms, hygiene, no smell, spotless bathrooms, fresh linens",
    "Comfort": "bed and pillow comfort, sleep quality, mattress quality",
    "Room Amenities": "wifi, tv, air conditioning, coffee maker, minibar, safe",
    "Bathroom": "shower water pressure, hot water, toiletries, towel quality",
    "Noise Level & Ambiance": "noise from hallways or outside, quiet rooms, lighting, curtains",
    "Space & Layout": "room size, storage, layout, power outlets and usb ports",

    "Check-in & Check-out": "reception speed, efficiency of check-in and check-out process",
    "Staff Professionalism & Attitude": "friendly or rude staff, helpful service, courteous behavior",
    "Responsiveness": "how fast staff respond to requests and problems",
    "Concierge Knowledge": "local recommendations, bookings, transportation help",
    "Housekeeping Efficiency": "daily cleaning quality, turndown service, timely housekeeping",

    "Proximity": "closeness to attractions and transport, walkable distance",
    "Neighborhood": "area safety, surrounding options for dining and shopping",
    "Accessibility & Parking": "car parking availability, ramps, elevators, wheelchair access",
    "The View": "room view, scenery from windows and common areas",

    "Restaurant Quality": "quality of food, taste, presentation, menu and service at restaurant",
    "Breakfast": "breakfast quality, freshness, variety, buffet or a la carte",
    "Room Service": "in-room dining, speed of delivery, food quality upon arrival",
    "Bar/Lounge": "bar ambiance, drink selection, bartender skill",

    "Recreational Facilities": "pool, spa, gym quality and cleanliness, operating hours",
    "Common Areas": "lobby and public area cleanliness, design and comfort",
    "Technology": "wifi speed and reliability across the hotel",
    "Business Services": "meeting rooms, business center, printing services",
    "Accessibility Features": "facilities for disabled guests, accessible rooms and elevators",

    "Value for Money": "price fairness for the overall experience",
    "Ambiance & Design": "hotel aesthetic, decor, atmosphere and design quality",
    "Safety & Security": "security staff presence, well lit areas, safe locks",
    "Sustainability Practices": "eco-friendly operations, recycling, reduced plastics",
}

ASPECTS = list(ASPECT_DESCRIPTIONS.keys())

def wordnet_synonyms(term: str, max_per_lemma=4):
    """Collect WordNet synonyms for key words inside an aspect name/description."""
    out = set()
    tokens = re.findall(r"[A-Za-z]+", term.lower())
    for t in tokens:
        for syn in wn.synsets(t):
            for l in syn.lemmas()[:max_per_lemma]:
                w = l.name().replace("_", " ")
                if len(w) > 2 and w.isascii():
                    out.add(w)
    return list(out)

# Build a prototype text for each aspect: title + description + WordNet expansions
aspect_prototypes = {}
for a in ASPECTS:
    base = ASPECT_DESCRIPTIONS[a]
    syns = wordnet_synonyms(a + " " + base)
    # keep it concise; too many synonyms can add noise
    syns = list(dict.fromkeys(syns))[:30]
    proto = f"{a}. {base}. Related terms: " + ", ".join(syns)
    aspect_prototypes[a] = proto

# Pre-compute prototype embeddings
aspect_proto_texts = [aspect_prototypes[a] for a in ASPECTS]
aspect_proto_emb = sbert.encode(aspect_proto_texts, normalize_embeddings=True)
print("Prepared prototype embeddings for", len(ASPECTS), "aspects.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Prepared prototype embeddings for 28 aspects.


*   Loads the all-MiniLM-L6-v2 model from sentence-transformers. This model is trained to convert sentences into dense 384-dimensional vectors (embeddings) where semantically similar sentences are close together in the vector space
*  This dictionary provides richer, more descriptive definitions for each aspect, which is much better for semantic matching than simple keywords.
*   def wordnet_synonyms(term: str, max_per_lemma=4):
•	This defines the function, which accepts two arguments:
o	term: The input string (e.g., "Cleanliness & Hygiene").
o	max_per_lemma: An optional argument that limits how many synonyms are collected for each meaning of a word. It defaults to 4.
*   out = set()
•	Initializes an empty set named out. A set is used because it automatically stores only unique items, preventing duplicate synonyms from being added to the final list.
*  tokens = re.findall(r"[A-Za-z]+", term.lower())
•	This line first converts the input term to lowercase (term.lower()).
•	Then, it uses a regular expression (re.findall) to find all sequences of alphabetic characters. This effectively splits the input string into a list of individual words, stripping away punctuation and symbols. For example, "Room Amenities" becomes ['room', 'amenities'].
*   for t in tokens:
•	This begins a loop that iterates through each word (t) in the tokens list.
*   for syn in wn.synsets(t):
•	For each token t, this line looks up all of its possible meanings in WordNet. Each meaning is represented by a synset (synonym set), which is a collection of words that are synonyms in a specific context. For example, the word "room" has synsets for "an area within a building" and for "opportunity for."
*   for l in syn.lemmas()[:max_per_lemma]:
•	For each synset, this line retrieves its associated lemmas (the specific words in the synonym set).
•	The [:max_per_lemma] part slices this list, taking only the first 4 (the default value) lemmas. This is a practical step to ensure that only the most common and relevant synonyms are collected, avoiding more obscure ones.
*  w = l.name().replace("_", " ")
•	This gets the string representation of the lemma. WordNet lemmas sometimes contain underscores for multi-word terms (e.g., 'living_room'). This line replaces the underscore with a space to create a more readable format ('living room').
*   if len(w) > 2 and w.isascii():
•	This is a filter to ensure data quality. It checks two conditions:
o	len(w) > 2: The synonym must be longer than two characters to filter out potentially meaningless short words.
o	w.isascii(): The synonym must contain only standard ASCII characters.
*   out.add(w)
•	If the synonym w passes the filter, it is added to the out set.
*   return list(out)
•	After all loops are complete, the function converts the set of unique synonyms into a list and returns it.
*   proto = f"{a}. {base}. Related terms: " + ", ".join(syns)
aspect_prototypes[a] = proto
o	For each aspect, it constructs a "prototype text" by combining the aspect's name, its description, and a list of synonyms generated from WordNet.
*  aspect_proto_emb = sbert.encode(aspect_proto_texts, normalize_embeddings=True)
o	The Sentence-BERT model is used to encode each of these prototype texts into a single, representative vector embedding. This creates a "semantic anchor" for each aspect in the vector space.


In [ ]:
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import pandas as pd
import numpy as np

SENT_MIN_LEN = 5
ASPECT_SIM_THRESHOLD = 0.38   # lower → more recall; higher → more precision (tune 0.35–0.45)
ASPECT_TOPK = 3               # cap number of aspects per sentence

def sentence_rows(df):
    rows = []
    for ridx, row in df.iterrows():
        sents = [s.strip() for s in sent_tokenize(str(row["text"])) if len(s.strip()) >= SENT_MIN_LEN]
        for s in sents:
            rows.append({"review_idx": int(ridx), "sentence": s})
    return pd.DataFrame(rows)

# 1) explode to sentence level
sent_df = sentence_rows(df)
print("Total sentences:", len(sent_df))

# 2) embed all sentences in batches
BATCH = 2048
all_sent_emb = []
for i in tqdm(range(0, len(sent_df), BATCH), desc="Embedding sentences"):
    embs = sbert.encode(sent_df["sentence"].iloc[i:i+BATCH].tolist(), normalize_embeddings=True)
    all_sent_emb.append(embs)
sent_emb = np.vstack(all_sent_emb)

# 3) cosine similarity to aspect prototypes
# sim: [num_sentences, num_aspects]
sim = np.matmul(sent_emb, aspect_proto_emb.T)

# 4) convert to multi-label aspect assignments
def aspects_for_row(sim_row):
    idx = np.argsort(-sim_row)  # desc
    chosen = []
    for j in idx[:ASPECT_TOPK]:
        if sim_row[j] >= ASPECT_SIM_THRESHOLD:
            chosen.append(ASPECTS[j])
    return chosen

sent_df["aspects"] = [aspects_for_row(sim[i]) for i in range(sim.shape[0])]
# keep only sentences with at least one aspect
sent_df = sent_df[sent_df["aspects"].map(len) > 0].reset_index(drop=True)

# 5) label sentence sentiment via your baseline (teacher)
import joblib
baseline = joblib.load("/content/drive/MyDrive/Hotels/Models/lr_sentiment.joblib")
proba = baseline.predict_proba(sent_df["sentence"])
classes = baseline.classes_
sent_df["sentiment"] = classes[proba.argmax(axis=1)]
sent_df["sent_conf"] = proba.max(axis=1)

# 6) explode to (sentence, aspect) rows
exploded = sent_df.explode("aspects").rename(columns={"aspects":"aspect"}).reset_index(drop=True)

# save for audit
sent_df.to_csv("/content/drive/MyDrive/Hotels/Outputs/sem_aspect_sentence_dataset.csv", index=False)
exploded.to_csv("/content/drive/MyDrive/Hotels/Outputs/sem_aspect_sentence_dataset_exploded.csv", index=False)
print("Saved ➜ outputs/sem_aspect_sentence_dataset(_exploded).csv")
print(exploded.head())


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Total sentences: 636886


Embedding sentences: 100%|██████████| 311/311 [03:45<00:00,  1.38it/s]


Saved ➜ outputs/sem_aspect_sentence_dataset(_exploded).csv
   review_idx                                           sentence  \
0           0                       Lovely ambience and service.   
1           0                       Lovely ambience and service.   
2           5           The staff was very kind to our daughter.   
3           7  Staff were super attentive and the rooms were ...   
4           7  Staff were super attentive and the rooms were ...   

                             aspect sentiment  sent_conf  
0                 Ambiance & Design  positive   0.881578  
1                    Room Amenities  positive   0.881578  
2  Staff Professionalism & Attitude  positive   0.892360  
3                      Room Service  positive   0.967568  
4  Staff Professionalism & Attitude  positive   0.967568  


*   sent_df = sentence_rows(df)
sent_emb = np.vstack(all_sent_emb)
o	First, all reviews are split into sentences and embedded into vectors using the SBERT model, just like the aspect prototypes

*  sim = np.matmul(sent_emb, aspect_proto_emb.T)
o	This is the key step. It performs a matrix multiplication between the sentence embeddings and the aspect prototype embeddings. For normalized vectors, this is a highly efficient way to calculate the cosine similarity between every sentence and every aspect. The result is a similarity score indicating how semantically related each sentence is to each aspect.

*   sent_df["aspects"] = [aspects_for_row(sim[i]) for i in range(sim.shape[0])]
o	The aspects_for_row function filters these similarities. For each sentence, it keeps only the aspects whose similarity score is above the ASPECT_SIM_THRESHOLD (0.38) and limits it to the top 3 matches (ASPECT_TOPK). This replaces the old keyword-based hint_aspects_for_sentence function with a much smarter, meaning-based one.


*  baseline = joblib.load(...)
sent_df["sentiment"] = classes[proba.argmax(axis=1)]
o	Just like before, the baseline "teacher" model is used to assign a weak sentiment label to these newly, and more accurately, aspect-tagged sentences. The resulting datasets (sem_aspect_sentence_dataset.csv, etc.) are saved.


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
import joblib, numpy as np

# train on sentences + multi-label aspects (from semantic mapping)
X_text = sent_df["sentence"].tolist()
Y_list = sent_df["aspects"].tolist()

mlb = MultiLabelBinarizer(classes=ASPECTS)
Y = mlb.fit_transform(Y_list)

X_train, X_test, Y_train, Y_test = train_test_split(
    X_text, Y, test_size=0.2, random_state=RANDOM_STATE
)

vec = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.95, sublinear_tf=True)
Xtr = vec.fit_transform(X_train)
Xte = vec.transform(X_test)

ovr = OneVsRestClassifier(
    LogisticRegression(max_iter=1000, class_weight="balanced", solver="lbfgs")
)
ovr.fit(Xtr, Y_train)

Y_pred = ovr.predict(Xte)
print("Aspect detector micro-F1:", f1_score(Y_test, Y_pred, average="micro"))
print("Aspect detector macro-F1:", f1_score(Y_test, Y_pred, average="macro"))

joblib.dump({"vec": vec, "clf": ovr, "mlb": mlb}, "/content/drive/MyDrive/Hotels/Models/aspect_detector_ovr.joblib")
print("Saved ➜ models/aspect_detector_ovr.joblib")


Aspect detector micro-F1: 0.8207368797588022
Aspect detector macro-F1: 0.7007248115539833
Saved ➜ models/aspect_detector_ovr.joblib


•	Cell 17 & 18: Re-training the Final Models
o	These cells are essentially repeats of Cells 9 and 10, but this time they are trained on the superior, semantically-labeled dataset generated by the BERT approach. This results in a more accurate Aspect Detector and more robust Per-Aspect Sentiment Models. The new, improved models are saved over the old ones.


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from collections import Counter
import joblib, os, re

def slugify(name: str):
    return re.sub(r'[^a-z0-9]+','_', name.lower()).strip('_')

# train per-aspect sentiment models
trained_aspect_models = {}
min_rows_per_aspect = 120
min_classes_per_aspect = 2

for aspect in ASPECTS:
    df_a = exploded[exploded["aspect"] == aspect].copy()
    if len(df_a) < min_rows_per_aspect or df_a["sentiment"].nunique() < min_classes_per_aspect:
        print(f"[skip] {aspect}: insufficient data (rows={len(df_a)}, classes={df_a['sentiment'].nunique()})")
        continue

    X_train, X_test, y_train, y_test = train_test_split(
        df_a["sentence"], df_a["sentiment"], test_size=0.2, random_state=RANDOM_STATE, stratify=df_a["sentiment"]
    )

    pipe = Pipeline([
        ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.95, sublinear_tf=True)),
        ("logreg", LogisticRegression(max_iter=1000, class_weight="balanced", solver="lbfgs"))
    ])
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)
    print(f"\n=== Sentiment student for [{aspect}] ===")
    print(classification_report(y_test, preds, digits=3))

    path = f"/content/drive/MyDrive/Hotels/Models/sentiment_aspect_{slugify(aspect)}.joblib"
    joblib.dump(pipe, path)
    trained_aspect_models[aspect] = path
    print("Saved ➜", path)

# general fallback (sentence-level)
gen_pipe = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.95, sublinear_tf=True)),
    ("logreg", LogisticRegression(max_iter=1000, class_weight="balanced", solver="lbfgs"))
])
gen_pipe.fit(sent_df["sentence"], sent_df["sentiment"])
joblib.dump(gen_pipe, "/content/drive/MyDrive/Hotels/Models/sentiment_sentence_student.joblib")
print("\nSaved general sentence sentiment student ➜ models/sentiment_sentence_student.joblib")

# ---------- Inspect: top tokens per aspect (helps you verify semantic coverage) ----------
def top_tfidf_terms_per_class(texts, labels, topn=20):
    vec = TfidfVectorizer(ngram_range=(1,2), min_df=5, max_df=0.9, sublinear_tf=True)
    X = vec.fit_transform(texts)
    terms = np.array(vec.get_feature_names_out())
    out = {}
    for aspect in ASPECTS:
        mask = (labels == aspect)
        if mask.sum() < 50:
            continue
        # class centroid: mean tfidf
        mean_vec = X[mask].mean(axis=0) - X[~mask].mean(axis=0)
        idx = np.asarray(mean_vec).ravel().argsort()[-topn:][::-1]
        out[aspect] = terms[idx].tolist()
    return out

# Build a (sentence, dominant_aspect) view for keyword mining
dominant = exploded.copy()
# choose the only aspect present for the sentence if unique; else skip to keep precision high
tmp = sent_df[sent_df["aspects"].map(len)==1].copy()
tmp["aspect"] = tmp["aspects"].str[0]
kw = top_tfidf_terms_per_class(tmp["sentence"].values, tmp["aspect"].values, topn=20)

with open("/content/drive/MyDrive/Hotels/Outputs/aspect_top_terms.json","w") as f:
    json.dump(kw, f, indent=2)
print("Saved aspect top terms ➜ outputs/aspect_top_terms.json")



=== Sentiment student for [Cleanliness & Hygiene] ===
              precision    recall  f1-score   support

    negative      0.681     0.814     0.742       113
     neutral      0.709     0.815     0.758       248
    positive      0.969     0.925     0.947      1292

    accuracy                          0.901      1653
   macro avg      0.786     0.851     0.815      1653
weighted avg      0.910     0.901     0.904      1653

Saved ➜ /content/drive/MyDrive/Hotels/Models/sentiment_aspect_cleanliness_hygiene.joblib

=== Sentiment student for [Comfort] ===
              precision    recall  f1-score   support

    negative      0.517     0.625     0.566        24
     neutral      0.602     0.878     0.714        74
    positive      0.974     0.891     0.931       459

    accuracy                          0.878       557
   macro avg      0.698     0.798     0.737       557
weighted avg      0.905     0.878     0.886       557

Saved ➜ /content/drive/MyDrive/Hotels/Models/sentimen

In [ ]:
import json
import pandas as pd
from tqdm import tqdm
import joblib
import numpy as np
import os, re
from nltk.tokenize import sent_tokenize

# ---------- Load models ----------
aspect_detect = joblib.load("/content/drive/MyDrive/Hotels/Models/aspect_detector_ovr.joblib")
aspect_vec = aspect_detect["vec"]
aspect_clf = aspect_detect["clf"]
aspect_mlb = aspect_detect["mlb"]

sent_student = joblib.load("/content/drive/MyDrive/Hotels/Models/sentiment_sentence_student.joblib")

def slugify(name: str):
    return re.sub(r'[^a-z0-9]+','_', name.lower()).strip('_')

aspect_models = {}
for a in aspect_mlb.classes_:
    p = f"/content/drive/MyDrive/Hotels/Models/sentiment_aspect_{slugify(a)}.joblib"
    if os.path.exists(p):
        aspect_models[a] = joblib.load(p)

# ---------- Helpers ----------
ASPECT_PROBA_THRESHOLD = 0.50
ASPECT_TOPK = 3
SENT_MIN_LEN = 5

def predict_aspects_for_sentence(sent: str):
    X = aspect_vec.transform([sent])
    proba = aspect_clf.predict_proba(X)[0]
    labels = aspect_mlb.classes_
    pairs = [(lab, float(p)) for lab, p in zip(labels, proba) if p >= ASPECT_PROBA_THRESHOLD]
    pairs.sort(key=lambda x: -x[1])
    if ASPECT_TOPK:
        pairs = pairs[:ASPECT_TOPK]
    return pairs

def predict_sentiment_for_sentence_aspect(sent: str, aspect: str):
    model = aspect_models.get(aspect, None)
    if model is None:
        label = sent_student.predict([sent])[0]
        conf = max(sent_student.predict_proba([sent])[0])
        return label, float(conf)
    else:
        label = model.predict([sent])[0]
        conf = max(model.predict_proba([sent])[0])
        return label, float(conf)

def analyze_review(text: str):
    sents = [s.strip() for s in sent_tokenize(str(text)) if len(s.strip()) >= SENT_MIN_LEN]
    aspect_map = {}
    for s in sents:
        asp_pairs = predict_aspects_for_sentence(s)
        if not asp_pairs:
            continue
        for (asp, _p) in asp_pairs:
            lab, conf = predict_sentiment_for_sentence_aspect(s, asp)
            prev = aspect_map.get(asp, {"votes": {"negative":0,"neutral":0,"positive":0}, "max_conf":0.0})
            prev["votes"][lab] += 1
            prev["max_conf"] = max(prev["max_conf"], conf)
            aspect_map[asp] = prev

    finalized = {}
    for asp, v in aspect_map.items():
        label = max(v["votes"].items(), key=lambda kv: kv[1])[0]
        finalized[asp] = {
            "sentiment": label,
            "hits": sum(v["votes"].values()),
            "conf": round(v["max_conf"], 3)
        }
    return finalized

# ---------- Run inference ----------
def run_absa_scikit(df: pd.DataFrame, max_reviews=None):
    work = df.copy()
    if max_reviews and len(work) > max_reviews:
        work = work.iloc[:max_reviews].copy()

    rows = []
    for i, row in tqdm(work.iterrows(), total=len(work), desc="ABSA (sklearn)"):
        text = str(row["text"])
        aspects = analyze_review(text)
        rows.append({
            "idx": i,
            "text": text,
            "aspects_json": json.dumps(aspects, ensure_ascii=False),
            "aspects_list": " | ".join(
                f"{a}={v['sentiment']} ({v['hits']} hits, conf≈{v['conf']:.2f})" for a, v in aspects.items()
            ) if aspects else "",
            "overall_label": row.get("overall_label", None)
        })

    out = pd.DataFrame(rows)
    out.to_csv("/content/drive/MyDrive/Hotels/Outputs/review_level_absa.csv", index=False)
    print("Saved ➜ outputs/review_level_absa.csv")

    # explode for summary
    long_rows = []
    for _, r in out.iterrows():
        d = json.loads(r["aspects_json"])
        for a, v in d.items():
            long_rows.append({
                "idx": r["idx"],
                "aspect": a,
                "sentiment": v["sentiment"],
                "hits": v["hits"],
                "conf": v["conf"],
            })

    if not long_rows:
        print("No aspect rows produced — check thresholds or training data.")
        return out, None, None

    long_df = pd.DataFrame(long_rows)
    long_df.to_csv("/content/drive/MyDrive/Hotels/Outputs/review_aspect_sentiments_long.csv", index=False)
    print("Saved ➜ outputs/review_aspect_sentiments_long.csv")

    summary = (long_df.groupby(["aspect","sentiment"])
               .size().reset_index(name="count")
               .pivot(index="aspect", columns="sentiment", values="count")
               .fillna(0).astype(int).reset_index().sort_values("aspect"))
    for col in ["negative","neutral","positive"]:
        if col not in summary.columns: summary[col] = 0
    summary = summary[["aspect","negative","neutral","positive"]]
    summary.to_csv("/content/drive/MyDrive/Hotels/Outputs/aspect_summary.csv", index=False)
    print("Saved ➜ outputs/aspect_summary.csv")

    return out, long_df, summary

# 🔹 run with a smaller batch first to test
review_level_absa, long_df, aspect_summary = run_absa_scikit(df, max_reviews=2000)

display(review_level_absa.head(3))
display(aspect_summary.head(10))


ABSA (sklearn): 100%|██████████| 2000/2000 [01:56<00:00, 17.13it/s]


Saved ➜ outputs/review_level_absa.csv
Saved ➜ outputs/review_aspect_sentiments_long.csv
Saved ➜ outputs/aspect_summary.csv


,idx,text,aspects_json,aspects_list,overall_label
0,0,Lovely ambience and service. I honestly couldn...,"{""Ambiance & Design"": {""sentiment"": ""positive""...","Ambiance & Design=positive (1 hits, conf≈0.85)...",positive
1,1,Top marks well earned. We stayed for four nigh...,"{""Noise Level & Ambiance"": {""sentiment"": ""posi...","Noise Level & Ambiance=positive (1 hits, conf≈...",positive
2,2,An Oasis. With a superb location it felt like ...,"{""Recreational Facilities"": {""sentiment"": ""pos...","Recreational Facilities=positive (1 hits, conf...",positive


sentiment,aspect,negative,neutral,positive
0,Accessibility & Parking,1,0,14
1,Accessibility Features,3,1,27
2,Ambiance & Design,27,5,516
3,Bar/Lounge,47,48,147
4,Bathroom,34,15,70
5,Breakfast,24,51,537
6,Business Services,20,16,183
7,Check-in & Check-out,35,13,30
8,Cleanliness & Hygiene,103,51,310
9,Comfort,24,19,163


## Cell 19: Final Inference Pipeline and Execution
This cell combines the logic from Cell 11 and Cell 12. It loads the final, BERT-trained models and runs the full end-to-end analysis on 2,000 reviews, saving the final CSV files to the Outputs directory. The logic inside the analyze_review function remains the same, but it now uses the much better models trained in the previous steps.


END OF **FIX**

In [1]:
!pip -q install streamlit altair


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 26.0 MB/s eta 0:00:00


•	Cell 20: Installs streamlit and altair, a library for creating beautiful declarative charts.

In [2]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

•	Cell 21: Downloads the NLTK punkt_tab tokenizer again, a small dependency for the Streamlit app's sentence splitting.

In [19]:
import pandas as pd
import re

# 1. Load your raw dataset (adjust path if needed)
RAW_PATH = "/content/drive/MyDrive/Hotels/hotels_reviews.csv"
df_raw = pd.read_csv(RAW_PATH)

# 2. Extract reviewer names and clean dates
def extract_reviewer(text):
    if not isinstance(text, str):
        return None
    parts = text.split("wrote a review")
    if len(parts) >= 2:
        return parts[0].strip()
    return None

def extract_date_part(text):
    if not isinstance(text, str):
        return None
    m = re.search(r"wrote a review\s+(.*)", text)
    if m:
        return m.group(1).strip()
    return None

df_raw["Reviewer_Name"] = df_raw["Review_Date"].apply(extract_reviewer)
df_raw["Review_Date_Cleaned"] = df_raw["Review_Date"].apply(extract_date_part)

# 3. Quick sanity check
print(df_raw[["Review_Date","Reviewer_Name","Review_Date_Cleaned"]].head())


                      Review_Date Reviewer_Name Review_Date_Cleaned
0  Rachel R wrote a review Mar 23      Rachel R              Mar 23
1   Ron N wrote a review Feb 2023         Ron N            Feb 2023
2  puddu101 wrote a review Mar 22      puddu101              Mar 22
3  tephrash wrote a review Mar 18      tephrash              Mar 18
4   David T wrote a review Mar 16       David T              Mar 16


In [26]:
import textwrap, json, os, pandas as pd

app_code = r"""
import json
import os
import pandas as pd
import altair as alt
import streamlit as st
import joblib
from pathlib import Path
from nltk.tokenize import sent_tokenize

st.set_page_config(page_title="Hotel ABSA Dashboard", layout="wide")

# ---------- Load data ----------
@st.cache_data
def load_csvs():
    base = Path("/content/drive/MyDrive/Hotels/Outputs")
    review_level = base / "review_level_absa.csv"
    aspect_long  = base / "review_aspect_sentiments_long.csv"
    aspect_sum   = base / "aspect_summary.csv"

    if not review_level.exists() or not aspect_long.exists() or not aspect_sum.exists():
        return None, None, None

    df_reviews = pd.read_csv(review_level)
    df_long    = pd.read_csv(aspect_long)
    df_sum     = pd.read_csv(aspect_sum)
    return df_reviews, df_long, df_sum

df_reviews, df_long, df_sum = load_csvs()

# Load raw dataset for meta-analytics (ratings, trip type, location, date)
RAW_PATH = "/content/drive/MyDrive/Hotels/hotels_reviews.csv"
df_raw = pd.read_csv(RAW_PATH)

# --- NEW: stable key + extract reviewer + clean dates ---
df_raw = df_raw.reset_index().rename(columns={"index": "idx"})

import re

def extract_reviewer(text):
    if not isinstance(text, str):
        return None
    parts = text.split("wrote a review")
    if len(parts) >= 2:
        return parts[0].strip()
    return None

def extract_date_part(text):
    if not isinstance(text, str):
        return None
    m = re.search(r"wrote a review\s+(.*)", text)
    if m:
        return m.group(1).strip()
    return None

# Add new columns
df_raw["Reviewer_Name"] = df_raw["Review_Date"].apply(extract_reviewer)
df_raw["Review_Date_Cleaned"] = df_raw["Review_Date"].apply(extract_date_part)

# Use cleaned date column in downstream parsing
df_raw["Cleaned_Review_Date"] = pd.to_datetime(df_raw["Review_Date_Cleaned"], errors="coerce")




import re
from datetime import datetime

_MONTHS3  = {m.lower(): i for i, m in enumerate(["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"], 1)}
_MONTHSFL = {m.lower(): i for i, m in enumerate(["January","February","March","April","May","June","July","August","September","October","November","December"], 1)}

def _month_to_int(name: str):
    if not isinstance(name, str): return None
    n = name.strip().lower()
    return _MONTHS3.get(n) or _MONTHSFL.get(n)

def _extract_year_from_text(s: str):

    if not isinstance(s, str): return None
    m = re.search(r'\b(19|20)\d{2}\b', s)
    return int(m.group(0)) if m else None

def _extract_month_year_from_stay(s: str):

    if not isinstance(s, str): return (None, None)
    m = re.search(r'([A-Za-z]{3,9})\s+((?:19|20)\d{2})', s)
    if not m: return (None, _extract_year_from_text(s))
    mon = _month_to_int(m.group(1))
    yr  = int(m.group(2))
    return (mon, yr)

def _parse_review_date_row(row):

    rd = row.get("Review_Date", "")
    stay = row.get("Reviewer_Date_Of_Stay", "")

    # 1) Month Day [, Year]   e.g., "Mar 23" or "Mar 23, 2021"
    m1 = re.search(r'([A-Za-z]{3,9})\s+(\d{1,2})(?:,\s*((?:19|20)\d{2}))?', str(rd))
    if m1:
        mon = _month_to_int(m1.group(1))
        day = int(m1.group(2))
        yr  = int(m1.group(3)) if m1.group(3) else None
        if yr is None:
            # Try to pull year from stay
            _, yr = _extract_month_year_from_stay(stay)
        if mon and yr:
            try: return pd.Timestamp(yr, mon, day)
            except Exception: return pd.NaT

    # 2) Month Year (no day)  e.g., "Feb 2023"
    m2 = re.search(r'([A-Za-z]{3,9})\s+((?:19|20)\d{2})', str(rd))
    if m2:
        mon = _month_to_int(m2.group(1))
        yr  = int(m2.group(2))
        if mon and yr:
            try: return pd.Timestamp(yr, mon, 1)
            except Exception: return pd.NaT

    # 3) Numeric dates like 2023-03-22, 22/03/2023, 03-22-2023
    m3 = re.search(r'\b(\d{4})[./-](\d{1,2})[./-](\d{1,2})\b', str(rd))  # YYYY-MM-DD
    if m3:
        try: return pd.Timestamp(int(m3.group(1)), int(m3.group(2)), int(m3.group(3)))
        except Exception: pass
    m4 = re.search(r'\b(\d{1,2})[./-](\d{1,2})[./-]((?:19|20)\d{2})\b', str(rd))  # DD-MM-YYYY or MM-DD-YYYY
    if m4:
        d1, d2, yr = int(m4.group(1)), int(m4.group(2)), int(m4.group(3))
        # Best effort: try DD-MM-YYYY first, then MM-DD-YYYY
        for day, mon in [(d1, d2), (d2, d1)]:
            try: return pd.Timestamp(yr, mon, day)
            except Exception: continue

    # 4) If only 'Mar 23' (no year) and stay has a year, use that year
    m5 = re.search(r'([A-Za-z]{3,9})\s+(\d{1,2})\b', str(rd))
    if m5:
        mon = _month_to_int(m5.group(1))
        day = int(m5.group(2))
        _, yr = _extract_month_year_from_stay(stay)
        if mon and yr:
            try: return pd.Timestamp(yr, mon, day)
            except Exception: return pd.NaT

    # 5) If only month in rd but year in stay (rare)
    m6 = re.search(r'\b([A-Za-z]{3,9})\b', str(rd))
    if m6:
        mon = _month_to_int(m6.group(1))
        smon, yr = _extract_month_year_from_stay(stay)
        if yr and (mon or smon):
            mon_final = mon or smon
            try: return pd.Timestamp(int(yr), int(mon_final), 1)
            except Exception: return pd.NaT

    return pd.NaT

# Build cleaned date column using BOTH fields
df_raw["Cleaned_Review_Date"] = df_raw.apply(_parse_review_date_row, axis=1)

# ---------- Load models ----------
def slugify(name: str):
    import re
    return re.sub(r'[^a-z0-9]+','_', name.lower()).strip('_')

aspect_detect = joblib.load("/content/drive/MyDrive/Hotels/Models/aspect_detector_ovr.joblib")
aspect_vec = aspect_detect["vec"]
aspect_clf = aspect_detect["clf"]
aspect_mlb = aspect_detect["mlb"]

sent_student = joblib.load("/content/drive/MyDrive/Hotels/Models/sentiment_sentence_student.joblib")
aspect_models = {}
ASPECTS = aspect_mlb.classes_.tolist()
for a in ASPECTS:
    p = f"/content/drive/MyDrive/Hotels/Models/sentiment_aspect_{slugify(a)}.joblib"
    if os.path.exists(p):
        aspect_models[a] = joblib.load(p)

ASPECT_PROBA_THRESHOLD = 0.50
ASPECT_TOPK = 3
SENT_MIN_LEN = 5

# ---------- Inference ----------
def predict_aspects_for_sentence(sent: str):
    X = aspect_vec.transform([sent])
    proba = aspect_clf.predict_proba(X)[0]
    labels = aspect_mlb.classes_
    pairs = [(lab, float(p)) for lab, p in zip(labels, proba) if p >= ASPECT_PROBA_THRESHOLD]
    pairs.sort(key=lambda x: -x[1])
    if ASPECT_TOPK:
        pairs = pairs[:ASPECT_TOPK]
    return pairs

def predict_sentiment_for_sentence_aspect(sent: str, aspect: str):
    model = aspect_models.get(aspect, None)
    if model is None:
        label = sent_student.predict([sent])[0]
        conf = max(sent_student.predict_proba([sent])[0])
        return label, float(conf)
    else:
        label = model.predict([sent])[0]
        conf = max(model.predict_proba([sent])[0])
        return label, float(conf)

def analyze_review(text: str):
    sents = [s.strip() for s in sent_tokenize(text) if len(s.strip()) >= SENT_MIN_LEN]
    aspect_map = {}
    for s in sents:
        asp_pairs = predict_aspects_for_sentence(s)
        if not asp_pairs:
            continue
        for (asp, _p) in asp_pairs:
            lab, conf = predict_sentiment_for_sentence_aspect(s, asp)
            prev = aspect_map.get(asp, {"votes": {"negative":0,"neutral":0,"positive":0}, "max_conf":0.0})
            prev["votes"][lab] += 1
            prev["max_conf"] = max(prev["max_conf"], conf)
            aspect_map[asp] = prev

    finalized = {}
    for asp, v in aspect_map.items():
        label = max(v["votes"].items(), key=lambda kv: kv[1])[0]
        finalized[asp] = {
            "sentiment": label,
            "hits": sum(v["votes"].values()),
            "conf": round(v["max_conf"], 3)
        }
    return finalized

# ---------- Dashboard ----------
st.title("🏨  LodgEdge ")
st.caption("Aspect-Based Sentiment Analysis + Hotel Insights")

# KPIs
if df_reviews is not None:
    total_reviews = len(df_reviews)
    total_aspect_rows = len(df_long) if df_long is not None else 0
    pos_rate = (df_long["sentiment"] == "positive").mean() if total_aspect_rows else 0
    neg_rate = (df_long["sentiment"] == "negative").mean() if total_aspect_rows else 0

    c1, c2, c3, c4 = st.columns(4)
    c1.metric("Total Reviews", f"{total_reviews:,}")
    c2.metric("Aspect Mentions", f"{total_aspect_rows:,}")
    c3.metric("Positive Share", f"{pos_rate*100:.1f}%")
    c4.metric("Negative Share", f"{neg_rate*100:.1f}%")

# ⭐ Overall Rating Distribution
if "Review_Rating" in df_raw.columns:
    st.subheader("⭐ Overall Rating Distribution")

    # 🔹 Normalize ratings: convert 10–50 scale → 1–5 scale
    df_raw["Normalized_Rating"] = df_raw["Review_Rating"].apply(
        lambda x: int(x/10) if (isinstance(x, (int,float)) and x > 5) else x
    )

    chart = alt.Chart(df_raw).mark_bar().encode(
        x=alt.X("Normalized_Rating:O", title="Rating (1–5)"),
        y=alt.Y("count()", title="Count of Reviews")
    )
    st.altair_chart(chart, use_container_width=True)


# 😊 Overall Sentiment Distribution
if "overall_label" in df_reviews.columns:
    st.subheader("😊 Overall Sentiment Distribution")
    dist = df_reviews["overall_label"].value_counts().reset_index()
    dist.columns = ["sentiment","count"]
    pie = alt.Chart(dist).mark_arc().encode(
        theta="count:Q",
        color="sentiment:N"
    )
    st.altair_chart(pie, use_container_width=True)

# 🧳 Reviewer Trip Type
if "Reviewer_Trip_Type" in df_raw.columns:
    st.subheader("🧳 Reviewer Trip Type Distribution")
    trip = df_raw["Reviewer_Trip_Type"].value_counts().reset_index()
    trip.columns = ["Trip Type","count"]
    chart = alt.Chart(trip).mark_bar().encode(x="Trip Type:N", y="count:Q")
    st.altair_chart(chart, use_container_width=True)

# 🌍 Reviewer Location
if "Reviewer_Location" in df_raw.columns:
    st.subheader("🌍 Reviewer Location Distribution (Top 15)")
    loc = df_raw["Reviewer_Location"].value_counts().reset_index().head(15)
    loc.columns = ["Location","count"]
    chart = alt.Chart(loc).mark_bar().encode(
        y=alt.Y("Location:N", sort="-x"), x="count:Q"
    )
    st.altair_chart(chart, use_container_width=True)

# 📊 Aspect Frequency
if df_long is not None:
    st.subheader("📊 Aspect Frequency Across Reviews")
    freq = df_long["aspect"].value_counts().reset_index()
    freq.columns = ["aspect","count"]
    chart = alt.Chart(freq).mark_bar().encode(
        y=alt.Y("aspect:N", sort="-x"), x="count:Q"
    )
    st.altair_chart(chart, use_container_width=True)

# 🔍 Sentiment per Aspect
if df_sum is not None:
    st.subheader("🔍 Sentiment Breakdown per Aspect")
    df_sum_melt = df_sum.melt(id_vars="aspect", value_vars=["negative","neutral","positive"],
                              var_name="sentiment", value_name="count")
    chart = alt.Chart(df_sum_melt).mark_bar().encode(
        y=alt.Y("aspect:N", sort="-x"), x="count:Q", color="sentiment:N"
    )
    st.altair_chart(chart, use_container_width=True)

# 💪 Strengths & Weaknesses (no overlap)
if df_sum is not None:
    st.subheader("💪 Strengths and Weaknesses")
    top_pos = df_sum.sort_values("positive", ascending=False).head(5)
    top_neg = df_sum[~df_sum["aspect"].isin(top_pos["aspect"])].sort_values("negative", ascending=False).head(5)

    c1, c2 = st.columns(2)
    c1.markdown("**Top 5 Strengths (Most Positive Aspects)**")
    pos_chart = alt.Chart(top_pos).mark_bar().encode(
        y=alt.Y("aspect:N", sort="-x"), x="positive:Q"
    )
    c1.altair_chart(pos_chart, use_container_width=True)

    c2.markdown("**Top 5 Weaknesses (Most Negative Aspects)**")
    neg_chart = alt.Chart(top_neg).mark_bar().encode(
        y=alt.Y("aspect:N", sort="-x"), x="negative:Q"
    )
    c2.altair_chart(neg_chart, use_container_width=True)

# 📈 Sentiment Over Time (robust: join on idx + cleaned dates + fallback sentiment if needed)
if ("Cleaned_Review_Date" in df_raw.columns) and ("idx" in df_raw.columns) and ("idx" in df_reviews.columns):
    st.subheader("📈 Sentiment Over Time")

    df_plot = df_reviews.copy()
    # Fallback if overall_label missing: derive from Review_Rating (10–50 or 1–5)
    if "overall_label" not in df_plot.columns or df_plot["overall_label"].isna().all():
        if "Review_Rating" in df_raw.columns:
            ratings = df_raw[["idx","Review_Rating"]].copy()
            df_plot = df_plot.merge(ratings, on="idx", how="left")
            def rate_to_label(x):
                try:
                    val = float(x)
                except Exception:
                    return None
                # normalize 10–50 to 1–5 if needed
                if val > 5:
                    val = val / 10.0
                if val <= 2:  return "negative"
                if val >= 4:  return "positive"
                return "neutral"
            df_plot["overall_label"] = df_plot["Review_Rating"].apply(rate_to_label)

    dates = df_raw[["idx","Cleaned_Review_Date"]].dropna(subset=["Cleaned_Review_Date"])
    merged = df_plot.merge(dates, on="idx", how="inner")

    if merged.empty or "overall_label" not in merged.columns:
        st.info("No valid dates or sentiments to plot.")
    else:
        merged["month"] = merged["Cleaned_Review_Date"].dt.to_period("M").dt.to_timestamp()
        timeline = (merged.dropna(subset=["month","overall_label"])
                          .groupby(["month","overall_label"])
                          .size().reset_index(name="count"))
        if timeline.empty:
            st.info("No data after cleaning dates. Check your Review_Date values.")
        else:
            chart = alt.Chart(timeline, width=4000, height=400).mark_line(point=True).encode(
                x=alt.X("month:T", title="Month"),
                y=alt.Y("count:Q", title="# Reviews"),
                color=alt.Color("overall_label:N", title="Sentiment",
                                sort=["negative","neutral","positive"])
            )
            st.altair_chart(chart, use_container_width=True)

# ---------- Try a New Review ----------
st.subheader("✍️ Try a New Review")
user_text = st.text_area("Paste a hotel review here:", height=120)

if st.button("Analyze Review") and user_text.strip():
    result = analyze_review(user_text)
    if not result:
        st.warning("No aspects detected in this review.")
    else:
        df_res = pd.DataFrame([
            {"aspect": a, "sentiment": v["sentiment"], "hits": v["hits"], "conf": v["conf"]}
            for a, v in result.items()
        ])
        st.dataframe(df_res, use_container_width=True)

        chart = alt.Chart(df_res).mark_bar().encode(
            y="aspect:N", x="hits:Q", color="sentiment:N"
        )
        st.altair_chart(chart, use_container_width=True)

# =========================
# 💼 Business Impact Insights
# (Append-only: does not change existing code)
# =========================
import numpy as np
import re

st.header("💼 Business Impact Insights")

# ---------- Helpers ----------
def _pick_col(df, patterns):
    pats = [re.compile(p, re.I) for p in patterns]
    for c in df.columns:
        for p in pats:
            if p.search(c):
                return c
    return None

def _safe_monthly_overall(df_reviews, df_raw):
    # Build monthly overall sentiment timeline (independent of earlier block scope)
    if not all(x in df_raw.columns for x in ["idx", "Cleaned_Review_Date"]):
        return None
    if "idx" not in df_reviews.columns:
        return None
    if "overall_label" not in df_reviews.columns:
        return None

    r = df_reviews[["idx", "overall_label"]].copy()
    d = df_raw[["idx", "Cleaned_Review_Date"]].dropna().copy()
    m = r.merge(d, on="idx", how="inner")
    if m.empty:
        return None
    m["month"] = m["Cleaned_Review_Date"].dt.to_period("M").dt.to_timestamp()
    return m

def _aspect_score_row(pos, neu, neg):
    tot = (pos or 0) + (neu or 0) + (neg or 0)
    if tot == 0:
        return 0.0
    # Score 0..100: weight neutral half
    return 100.0 * (pos + 0.5 * neu) / tot

def _verdict(pct):
    if pct >= 85: return "excellent"
    if pct >= 70: return "good"
    if pct >= 50: return "needs work"
    return "bad"

def _find_reviewer_col(df):
    return _pick_col(df, [
        r"^reviewer.*name$", r"reviewer$",
        r"^guest.*name$", r"^author$", r"user(name)?$", r"customer.*name"
    ])

def _find_hotel_col(df):
    return _pick_col(df, [r"hotel", r"property", r"resort"])

# ---------- Tabs ----------
tabs = st.tabs([
    "Aspect Performance",
    "Per-Aspect Trends",
    "Best Months",
    "Year Overview",
    "Aspect–Revenue Correlation (Synthetic)",
    "Guest Retention & Loyalty",
    "Upsell & Cross-Sell",
    "Early-Warning Alerts",
    "Revenue Impact Forecasting (Synthetic)",
    "Competitor Benchmarking"
])

# -------------------------
# 1) Aspect Performance
# -------------------------
with tabs[0]:
    st.subheader("Aspect Performance Score")
    if df_sum is not None and set(["aspect","positive","neutral","negative"]).issubset(df_sum.columns):
        perf = df_sum.copy()
        perf["score_pct"] = perf.apply(lambda r: _aspect_score_row(r.get("positive",0), r.get("neutral",0), r.get("negative",0)), axis=1)
        perf["verdict"] = perf["score_pct"].round(1).apply(_verdict)
        perf = perf.sort_values("score_pct", ascending=False).reset_index(drop=True)

        st.dataframe(perf[["aspect","positive","neutral","negative","score_pct","verdict"]], use_container_width=True)

        bar = alt.Chart(perf).mark_bar().encode(
            y=alt.Y("aspect:N", sort="-x", title="Aspect"),
            x=alt.X("score_pct:Q", title="Performance Score (0–100)"),
            color=alt.Color("verdict:N", sort=["bad","needs work","good","excellent"])
        )
        st.altair_chart(bar, use_container_width=True)
        st.caption("Score = (Positive + 0.5×Neutral) / Total × 100. Verdict thresholds: ≥85 excellent; 70–84 good; 50–69 needs work; <50 bad.")
    else:
        st.info("`df_sum` with columns ['aspect','positive','neutral','negative'] not found.")

# -------------------------
# 2) Per-Aspect Trends
# -------------------------
with tabs[1]:
    st.subheader("Time-Aware Sentiment per Aspect (Monthly)")
    # We need df_long with aspect+sentiment and idx to join dates
    if df_long is not None and "sentiment" in df_long.columns and "aspect" in df_long.columns:
        if ("idx" in df_long.columns) and ("idx" in df_raw.columns) and ("Cleaned_Review_Date" in df_raw.columns):
            dl = df_long[["idx","aspect","sentiment"]].merge(
                df_raw[["idx","Cleaned_Review_Date"]].dropna(), on="idx", how="inner"
            )
        elif "idx" in df_raw.columns and "idx" not in df_long.columns:
            # Best-effort positional align (fallback only)
            dl = pd.concat([df_long.reset_index(drop=True), df_raw[["idx","Cleaned_Review_Date"]].dropna().reset_index(drop=True)], axis=1)
            if "Cleaned_Review_Date" not in dl.columns:
                st.info("Unable to align aspects to dates; `idx` missing in df_long.")
                dl = None
        else:
            dl = None

        if dl is not None and not dl.empty and "Cleaned_Review_Date" in dl.columns:
            dl["month"] = pd.to_datetime(dl["Cleaned_Review_Date"]).dt.to_period("M").dt.to_timestamp()
            aspects = sorted(dl["aspect"].dropna().unique().tolist())
            pick = st.multiselect("Choose aspects to plot", options=aspects, default=aspects[:5])
            if pick:
                sub = dl[dl["aspect"].isin(pick)].copy()
                timeline = (sub.groupby(["month","aspect","sentiment"]).size().reset_index(name="count"))
                chart = alt.Chart(timeline).mark_line(point=True).encode(
                    x=alt.X("month:T", title="Month"),
                    y=alt.Y("count:Q", title="# Mentions"),
                    color="sentiment:N",
                    facet=alt.Facet("aspect:N", columns=2),
                    tooltip=["month:T","aspect:N","sentiment:N","count:Q"]
                )
                st.altair_chart(chart, use_container_width=True)
            else:
                st.info("Select at least one aspect.")
        else:
            st.info("Could not build per-aspect timeline (missing dates or idx).")
    else:
        st.info("`df_long` with ['aspect','sentiment'] not found.")

# -------------------------
# 3) Best Performing Months
# -------------------------
with tabs[2]:
    st.subheader("Best Performing Months (Overall Positivity)")
    m = _safe_monthly_overall(df_reviews, df_raw)
    if m is not None:
        agg = (m.groupby("month")["overall_label"]
                 .value_counts()
                 .unstack(fill_value=0)
                 .reset_index())
        for c in ["negative","neutral","positive"]:
            if c not in agg.columns: agg[c] = 0
        agg["total"] = agg[["negative","neutral","positive"]].sum(axis=1)
        agg["positivity_rate"] = (agg["positive"] / agg["total"]).round(4)
        topn = agg.sort_values("positivity_rate", ascending=False).head(10)
        st.dataframe(topn[["month","positive","neutral","negative","total","positivity_rate"]], use_container_width=True)

        chart = alt.Chart(agg).mark_line(point=True).encode(
            x="month:T", y=alt.Y("positivity_rate:Q", title="Positivity Rate"),
            tooltip=["month:T","positivity_rate:Q"]
        )
        st.altair_chart(chart, use_container_width=True)
    else:
        st.info("Monthly overall sentiment not available.")

# -------------------------
# 4) Year Overview
# -------------------------
with tabs[3]:
    st.subheader("Year Overview & Positivity Rate")
    m = _safe_monthly_overall(df_reviews, df_raw)
    if m is not None:
        m["year"] = m["month"].dt.year
        yr = (m.groupby("year")["overall_label"].value_counts()
                .unstack(fill_value=0).reset_index())
        for c in ["negative","neutral","positive"]:
            if c not in yr.columns: yr[c] = 0
        yr["total"] = yr[["negative","neutral","positive"]].sum(axis=1)
        yr["positivity_rate"] = (yr["positive"] / yr["total"]).round(4)
        st.dataframe(yr[["year","positive","neutral","negative","total","positivity_rate"]],
                     use_container_width=True)

        chart = alt.Chart(yr).mark_bar().encode(
            x=alt.X("year:O", title="Year"),
            y=alt.Y("positive:Q", title="# Positive"),
            tooltip=["year:O","positive:Q","neutral:Q","negative:Q","positivity_rate:Q"]
        )
        st.altair_chart(chart, use_container_width=True)
    else:
        st.info("Year overview not available.")

# -----------------------------------------
# 5) Aspect–Revenue Correlation (Synthetic)
# -----------------------------------------
with tabs[4]:
    st.subheader("Aspect–Revenue Correlation (Synthetic KPIs)")
    # Build monthly per-aspect positivity
    if df_long is not None and "sentiment" in df_long.columns and "aspect" in df_long.columns and "idx" in df_long.columns:
        join = df_long[["idx","aspect","sentiment"]].merge(
            df_raw[["idx","Cleaned_Review_Date"]].dropna(), on="idx", how="inner"
        )
        if not join.empty:
            join["month"] = pd.to_datetime(join["Cleaned_Review_Date"]).dt.to_period("M").dt.to_timestamp()
            aspect_list = sorted(join["aspect"].dropna().unique().tolist())
            a_sel = st.selectbox("Aspect", options=aspect_list, index=0)

            sub = join[join["aspect"] == a_sel]
            month_counts = (sub.groupby(["month","sentiment"]).size().unstack(fill_value=0).reset_index())
            for c in ["negative","neutral","positive"]:
                if c not in month_counts.columns: month_counts[c] = 0
            month_counts["total"] = month_counts[["negative","neutral","positive"]].sum(axis=1)
            month_counts = month_counts[month_counts["total"] > 0].copy()
            month_counts["pos_share"] = month_counts["positive"] / month_counts["total"]

            # Synthetic KPIs correlated with pos_share
            rng = np.random.default_rng(42)
            base_ADR = 80 + 70 * month_counts["pos_share"].values + rng.normal(0, 5, size=len(month_counts))
            conv_rate = 0.10 + 0.25 * month_counts["pos_share"].values + rng.normal(0, 0.01, size=len(month_counts))
            month_counts["synthetic_ADR"] = base_ADR.round(2)
            month_counts["synthetic_Conversion"] = np.clip(conv_rate, 0, 1).round(3)

            # Correlations
            corr_adr = float(np.corrcoef(month_counts["pos_share"], month_counts["synthetic_ADR"])[0,1])
            corr_conv = float(np.corrcoef(month_counts["pos_share"], month_counts["synthetic_Conversion"])[0,1])

            st.caption(f"Correlation with ADR: **{corr_adr:.2f}**, with Conversion: **{corr_conv:.2f}** (synthetic).")

            sc1 = alt.Chart(month_counts).mark_circle(size=80).encode(
                x=alt.X("pos_share:Q", title="Positive Share (Aspect)"),
                y=alt.Y("synthetic_ADR:Q", title="ADR (Synthetic)"),
                tooltip=["month:T","pos_share:Q","synthetic_ADR:Q"]
            )
            sc1 = sc1 + sc1.transform_regression("pos_share","synthetic_ADR").mark_line()
            st.altair_chart(sc1, use_container_width=True)

            sc2 = alt.Chart(month_counts).mark_circle(size=80).encode(
                x=alt.X("pos_share:Q", title="Positive Share (Aspect)"),
                y=alt.Y("synthetic_Conversion:Q", title="Conversion Rate (Synthetic)"),
                tooltip=["month:T","pos_share:Q","synthetic_Conversion:Q"]
            )
            sc2 = sc2 + sc2.transform_regression("pos_share","synthetic_Conversion").mark_line()
            st.altair_chart(sc2, use_container_width=True)
        else:
            st.info("Could not compute per-aspect monthly sentiment (missing idx/date join).")
    else:
        st.info("`df_long` with ['idx','aspect','sentiment'] required.")

# -------------------------------
# 6) Guest Retention & Loyalty
# -------------------------------
with tabs[5]:
    st.subheader("Guest Retention & Loyalty Metrics")

    # --- try auto-detect first ---
    rid = _find_reviewer_col(df_raw)

    # --- interactive fallback if auto-detect failed ---
    if rid is None:
        st.info("Pick the column that identifies the reviewer (name, username, profile id, etc.).")
        # candidates: text-like columns with some repetition (to capture repeat guests)
        obj_cols = [c for c in df_raw.columns if df_raw[c].dtype == "object"]
        candidates = []
        for c in obj_cols:
            s = df_raw[c].dropna().astype(str)
            if len(s) >= 20:
                uniq = s.nunique()
                # we want some duplicates (repeat guests), but not all unique
                if 0 < uniq < len(s):
                    candidates.append((c, len(s) - uniq))
        # sort by number of duplicates (more repeats first)
        candidates = [c for c, _ in sorted(candidates, key=lambda x: -x[1])] or obj_cols

        rid = st.selectbox("Reviewer identifier column", options=candidates)

    if rid is None:
        st.info("No suitable text column found for reviewer id.")
    else:
        base = df_raw[[rid, "idx"]].dropna()
        if "idx" in df_reviews.columns and "overall_label" in df_reviews.columns:
            guests = base.merge(df_reviews[["idx", "overall_label"]], on="idx", how="left")

            # Repeat guests
            counts = guests.groupby(rid).size().rename("reviews").reset_index()
            repeat = counts[counts["reviews"] >= 2]
            st.metric("Repeat Guests", f"{len(repeat):,}")

            # Positivity per guest + dummy rebooking likelihood
            gpos = (guests.groupby(rid)["overall_label"]
                        .value_counts().unstack(fill_value=0).reset_index())
            for c in ["negative", "neutral", "positive"]:
                if c not in gpos.columns: gpos[c] = 0
            gpos["total"] = gpos[["negative","neutral","positive"]].sum(axis=1)
            gpos = gpos[gpos["total"] > 0].copy()
            gpos["pos_rate"] = gpos["positive"] / gpos["total"]

            def rebook_prob(p):
                return float(np.clip(0.2 + 0.7 * p, 0.0, 0.95))

            gpos["rebook_likelihood"] = gpos["pos_rate"].apply(rebook_prob)

            st.dataframe(
                gpos.sort_values("rebook_likelihood", ascending=False)
                    [[rid, "positive", "neutral", "negative", "total", "pos_rate", "rebook_likelihood"]]
                    .head(50),
                use_container_width=True
            )

            chart = alt.Chart(gpos).mark_bar().encode(
                x=alt.X("rebook_likelihood:Q", bin=alt.Bin(maxbins=20), title="Rebooking Likelihood (dummy)"),
                y="count()"
            )
            st.altair_chart(chart, use_container_width=True)
        else:
            st.info("Need df_reviews with ['idx','overall_label'] to compute loyalty metrics.")


# -------------------------------
# 7) Upsell & Cross-Sell Ideas
# -------------------------------
with tabs[6]:
    st.subheader("Upsell & Cross-Sell Opportunities")
    if df_sum is not None and {"aspect","positive"}.issubset(df_sum.columns):
        top_pos = df_sum.sort_values("positive", ascending=False).head(10)["aspect"].tolist()
        st.write("Top positively-mentioned aspects:", ", ".join(top_pos))
        # Simple suggestions
        suggestions = {
            "Breakfast": "Promote premium breakfast/brunch packages.",
            "Food": "Chef’s tasting menu add-on at booking.",
            "Spa": "Spa credit bundles with stay.",
            "Pool": "Poolside cabana/day-pass upsell.",
            "Staff": "VIP check-in / concierge upsell.",
            "Cleanliness": "Premium housekeeping / late checkout add-on.",
            "Location": "Local tour packages & transfers.",
            "Price": "Bundle offers (stay + meal) to lift perceived value."
        }
        for a in top_pos:
            idea = suggestions.get(a, "Create an add-on or bundle highlighting this strength.")
            st.markdown(f"- **{a}** → {idea}")
    else:
        st.info("`df_sum` with aspect sentiment totals is required.")

# --------------------------------
# 8) Early-Warning Alerts (Risk)
# --------------------------------
with tabs[7]:
    st.subheader("Early-Warning Alerts → Revenue Risk")
    # Use per-aspect month over month change
    if df_long is not None and "idx" in df_long.columns and "sentiment" in df_long.columns and "aspect" in df_long.columns:
        j = df_long[["idx","aspect","sentiment"]].merge(
            df_raw[["idx","Cleaned_Review_Date"]].dropna(), on="idx", how="inner"
        )
        if not j.empty:
            j["month"] = pd.to_datetime(j["Cleaned_Review_Date"]).dt.to_period("M").dt.to_timestamp()
            # positive rate per aspect per month
            g = (j.groupby(["aspect","month"])["sentiment"]
                   .value_counts().unstack(fill_value=0).reset_index())
            for c in ["negative","neutral","positive"]:
                if c not in g.columns: g[c] = 0
            g["total"] = g[["negative","neutral","positive"]].sum(axis=1)
            g = g[g["total"]>0].copy()
            g["pos_rate"] = g["positive"]/g["total"]

            # Compare last 2 months that exist
            last_months = sorted(g["month"].unique())
            if len(last_months) >= 2:
                m1, m0 = last_months[-1], last_months[-2]
                cur = g[g["month"]==m1][["aspect","pos_rate"]].set_index("aspect")
                prev = g[g["month"]==m0][["aspect","pos_rate"]].set_index("aspect")
                chg = (cur["pos_rate"] - prev["pos_rate"]).sort_values()
                threshold = st.slider("Alert threshold (percentage points drop)", 5, 30, 12)
                for a, delta in chg.items():
                    drop_pp = delta * 100
                    if drop_pp <= -threshold:
                        est_risk = abs(drop_pp) * 0.6  # dummy mapping
                        st.error(f"**{a}** dropped {abs(drop_pp):.1f}pp (from {prev.loc[a,'pos_rate']*100:.1f}% to {cur.loc[a,'pos_rate']*100:.1f}%). Potential revenue risk ≈ {est_risk:.1f}%.")
                if (chg*100 >= -threshold).all():
                    st.success("No significant aspect drops detected.")
            else:
                st.info("Need at least two months of aspect data to compute alerts.")
        else:
            st.info("Could not align aspects to dates for alerts.")
    else:
        st.info("`df_long` with ['idx','aspect','sentiment'] required.")

# --------------------------------
# 9) Revenue Impact Forecasting (Synthetic)
# --------------------------------
with tabs[8]:
    st.subheader("Revenue Impact Forecasting (Synthetic)")
    if df_sum is not None and {"aspect","positive","neutral","negative"}.issubset(df_sum.columns):
        base = df_sum.copy()
        base["score_pct"] = base.apply(lambda r: _aspect_score_row(r.get("positive",0), r.get("neutral",0), r.get("negative",0)), axis=1)
        aspects = base["aspect"].tolist()
        a_sel = st.selectbox("Aspect to improve", options=aspects, index=0)
        improve = st.slider("Improvement in aspect score (pp)", 0, 40, 20)

        cur = float(base.loc[base["aspect"]==a_sel, "score_pct"].values[0])
        new = np.clip(cur + improve, 0, 100)

        # Simple synthetic elasticity: every +10pp → +4% RevPAR
        uplift_pct = (new - cur) / 10.0 * 4.0
        # Synthetic current RevPAR ($)
        rng = np.random.default_rng(7)
        cur_revpar = float(100 + rng.normal(0, 5))
        horizon = st.selectbox("Horizon", ["3 months","6 months","12 months"], index=1)
        mult = {"3 months": 0.5, "6 months": 1.0, "12 months": 2.0}[horizon]
        proj_rev_gain = cur_revpar * (uplift_pct/100.0) * mult

        c1, c2, c3 = st.columns(3)
        c1.metric("Current score", f"{cur:.1f}%")
        c2.metric("New score", f"{new:.1f}%")
        c3.metric("Projected RevPAR Gain", f"${proj_rev_gain:,.0f}")

        line = pd.DataFrame({
            "Scenario": ["Current","Improved"],
            "Score": [cur, new]
        })
        ch = alt.Chart(line).mark_bar().encode(
            x="Scenario:N", y=alt.Y("Score:Q", title="Aspect Score (%)")
        )
        st.altair_chart(ch, use_container_width=True)
        st.caption("Synthetic model: +10pp in aspect score → ~+4% RevPAR; horizon scales impact. Replace with real KPIs when available.")
    else:
        st.info("`df_sum` with aspect totals required for forecasting baseline.")

# --------------------------------
# 10) Competitor Benchmarking
# --------------------------------
with tabs[9]:
    st.subheader("Competitor Benchmarking (Across Hotels)")
    hcol = _find_hotel_col(df_raw)
    if hcol is None:
        st.info("Could not find a hotel/property column in df_raw.")
    else:
        # overall positivity by hotel
        m = _safe_monthly_overall(df_reviews, df_raw)
        if m is not None:
            hotels = df_raw[["idx", hcol]]
            mh = m.merge(hotels, on="idx", how="left").dropna(subset=[hcol])
            agg = (mh.groupby(hcol)["overall_label"].value_counts()
                     .unstack(fill_value=0).reset_index())
            for c in ["negative","neutral","positive"]:
                if c not in agg.columns: agg[c] = 0
            agg["total"] = agg[["negative","neutral","positive"]].sum(axis=1)
            agg = agg[agg["total"]>0].copy()
            agg["positivity_rate"] = (agg["positive"]/agg["total"]).round(4)

            # Choose primary hotel (from page title if present)
            default_hotel = None
            for v in df_raw[hcol].value_counts().index:
                if "savoy" in str(v).lower(): default_hotel = v; break

            target = st.selectbox("Your hotel", options=agg[hcol].tolist(),
                                  index=(agg[hcol].tolist().index(default_hotel) if default_hotel in agg[hcol].tolist() else 0))
            topn = st.slider("Show top N hotels by positivity", 5, 30, 10)

            rank = agg.sort_values("positivity_rate", ascending=False).head(topn)
            chart = alt.Chart(rank).mark_bar().encode(
                x=alt.X("positivity_rate:Q", title="Positivity Rate"),
                y=alt.Y(f"{hcol}:N", sort="-x", title="Hotel"),
                tooltip=[f"{hcol}:N","positivity_rate:Q","positive:Q","neutral:Q","negative:Q","total:Q"]
            )
            st.altair_chart(chart, use_container_width=True)

            # Highlight your hotel
            me = agg[agg[hcol]==target]
            if not me.empty:
                pr = float(me["positivity_rate"].values[0])*100
                st.metric(f"Positivity rate — {target}", f"{pr:.1f}%")
        else:
            st.info("Could not compute overall sentiment per hotel (missing idx/date join).")

"""

with open("app.py","w",encoding="utf-8") as f:
    f.write(app_code)

print("Wrote updated app.py ✅ (timeline fixed: idx join + date cleaning)")


Wrote updated app.py ✅ (timeline fixed: idx join + date cleaning)


•	Cell 22: Generating the app.py Script
*   This entire cell is a single, large string app_code that contains the complete Python code for a Streamlit application.
*   with open("app.py","w",...) as f: f.write(app_code): This writes the string to a file named app.py in the Colab environment.
Inside the app.py code:
*   Data Loading: It loads the output CSVs (review_level_absa.csv, etc.) and the original raw dataset.
*   Date Cleaning: It includes a robust set of functions to parse the inconsistent date formats found in the raw data, creating a clean Cleaned_Review_Date column for time-series analysis.
*   Model Loading: It loads all the final trained models (aspect_detector_ovr.joblib, per-aspect models, etc.) needed for live inference.
*   Inference Functions: The analyze_review function is copied here so the app can analyze new, user-submitted text in real-time.
*   UI Layout: The rest of the code uses st. commands to build the web interface:
	st.title, st.metric: Creates the title and key performance indicator (KPI) cards at the top.
	st.altair_chart: Renders various charts using the Altair library to visualize distributions, aspect frequencies, strengths, and weaknesses.
	st.tabs: Creates the "Business Impact Insights" section with multiple tabs for different analyses like revenue correlation and guest retention.
	st.text_area, st.button: Creates the interactive text box and button for users to analyze their own reviews.


In [4]:
!pip -q install pyngrok streamlit


•	Cell 23: Installs pyngrok, a Python wrapper for ngrok. ngrok is a service that creates a secure tunnel from a public endpoint (a URL) to a locally running web service (the Streamlit app inside Colab).

In [5]:
from pyngrok import ngrok

# 🔑 replace with your token (only once per session)
NGROK_AUTH_TOKEN = "2yKhsIeQh6ZTvNbzroSPiAKRWFR_3FA8jmFRAjyP4DvVUo4C1"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)


•	Cell 24: Sets the authentication token for your ngrok account. This is required to use the service.

In [6]:
import threading, time, os
from pyngrok import ngrok

# Kill any existing streamlit processes
os.system("pkill -f 'streamlit run' >/dev/null 2>&1 || true")

def run_streamlit():
    os.system("streamlit run app.py --server.port 8501 --server.address 0.0.0.0")

# Start streamlit in background
t = threading.Thread(target=run_streamlit, daemon=True)
t.start()

# Give streamlit time to boot
time.sleep(5)

# Open ngrok tunnel
public_url = ngrok.connect(8501)
print("🌐 Streamlit app running at:", public_url)
print("⚠️ Keep this cell running; stop = tunnel closes")


🌐 Streamlit app running at: NgrokTunnel: "https://e48ae3ec895a.ngrok-free.app" -> "http://localhost:8501"
⚠️ Keep this cell running; stop = tunnel closes


•	Cell 25: Running and Exposing the App

*   os.system("pkill -f 'streamlit run' ...")
o	Kills any previously running Streamlit processes to ensure a fresh start.
*  t = threading.Thread(target=run_streamlit, daemon=True)
t.start()
o	The command streamlit run app.py is executed in a separate background thread. This starts the web server without blocking the notebook cell from continuing.
*   public_url = ngrok.connect(8501)
print("🌐 Streamlit app running at:", public_url)
*   ngrok.connect(8501) tells ngrok to create a public URL and forward all traffic to port 8501 on the local machine, where Streamlit is running.
o	The public URL is then printed. Clicking this link opens the interactive dashboard in your browser. The cell must remain running to keep the tunnel open.
